In [1]:
# import tensorflow as tf
# from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
# from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertConfig
# from transformers import TFBertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

In [2]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch



In [3]:
import pandas as pd

pos_texts = pd.read_csv('positive.csv', encoding='utf8', sep=';', header=None)
neg_texts = pd.read_csv('negative.csv', encoding='utf8', sep=';', header=None)

In [4]:
pos_texts.sample(10)

,0,1,2,3,4,5,6,7,8,9,10,11
29980,409633231915741185,1386499147,esacujeqs,RT @yganumoz: Жарко? А я намазалась кремом с м...,1,0,3,0,196,127,120,0
14597,409339264863047680,1386429060,skafimerca,"RT @bankpiho: Я вся такая легкая, такая воздуш...",1,0,16,0,607,230,192,0
52039,410051095176564736,1386598774,menageatrois98,"@HeeyDoctor по сути даа..\nнас же много таких,...",1,0,0,0,7246,276,267,1
84206,410782686903422976,1386773199,SanyaPro_1988,@Belkinparadise @Lisinparadise Хз ну белка по...,1,0,0,0,1000,13,9,0
52435,410054216510476288,1386599518,medich4141,@JuliaVolkova @Zvezdyvtwi Пуськи уже выступили...,1,0,0,0,17638,907,1221,14
77583,410711137362399232,1386756140,oriroora,"Ты сволочь, всю молодость мне испортил! Я тебе...",1,0,0,0,1584,278,494,0
101916,411097669990641664,1386848297,Vad_Im,@milkvi у этих ребят даже будут работать с рос...,1,0,0,0,28708,497,241,35
18995,409385984745570304,1386440199,pyqyhawyfoge,RT @wyhevovelyjo: ительность действительно сни...,1,0,1,0,388,198,179,0
81550,410757673164881920,1386767235,BogE92,пойти поспать часика полтора и пора выезжать н...,1,0,0,0,777,534,189,0
12466,409312724741787648,1386422732,vikhyun_oppa,Налегла на Кэти и Тейлор в последнее время. Во...,1,0,0,0,168,24,31,0


Обратите внимание на специальные токены [CLS] и [SEP], которые мы добавляем в началои конец предложения.

In [5]:
sentences = np.concatenate([pos_texts[3].values, neg_texts[3].values])

sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
labels = [[1] for _ in range(pos_texts.shape[0])] + [[0] for _ in range(neg_texts.shape[0])]

In [6]:
assert len(sentences) == len(labels) == pos_texts.shape[0] + neg_texts.shape[0]

In [7]:
print(sentences[1000])

[CLS] Дим, ты помогаешь мне, я тебе, все взаимно, все правильно) [SEP]


In [8]:
from sklearn.model_selection import train_test_split

train_sentences, test_sentences, train_gt, test_gt = train_test_split(sentences, labels, test_size=0.3)

In [9]:
print(len(train_gt), len(test_gt))

158783 68051


Проводим токинезацию по бертовски!

In [10]:
%%time
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

tokenized_texts = [tokenizer.tokenize(sent) for sent in train_sentences]
print (tokenized_texts[0])

['[CLS]', 'г', '##е', '##р', '##а', ',', 'у', '##и', '##д', '##и', 'о', '##т', '##с', '##ю', '##д', '##а', '.', 'я', 'и', '##т', '##а', '##к', 'у', '##с', '##н', '##у', '##л', '##а', 'р', '##ан', '##ь', '##ш', '##е', ',', 'ч', '##т', '##о', '##б', '##ы', 'т', '##е', '##б', '##я', 'н', '##е', 'б', '##ы', '##л', '##о', 'н', '##о', '##ч', '##ь', '##ю', '.', 'к', '##а', '##к', '##о', '##г', '##о', 'в', '##о', '##о', '##б', '##щ', '##е', 'и', '##ии', '##ии', 'т', '##ы', 'и', '##г', '##р', '##а', '##е', '##ш', '##ь', 'в', 'м', '##о', '##е', '##и', 'к', '##о', '##м', '##на', '##т', '##е', 'н', '##о', '##ч', '##ь', '##ю', '?', '!', '&', 'gt', ';', ':', '(', '(', '(', '[SEP]']
CPU times: user 1min 24s, sys: 1.47 s, total: 1min 25s
Wall time: 1min 31s


In [16]:
tokenized_texts[0]

['[CLS]',
 'г',
 '##е',
 '##р',
 '##а',
 ',',
 'у',
 '##и',
 '##д',
 '##и',
 'о',
 '##т',
 '##с',
 '##ю',
 '##д',
 '##а',
 '.',
 'я',
 'и',
 '##т',
 '##а',
 '##к',
 'у',
 '##с',
 '##н',
 '##у',
 '##л',
 '##а',
 'р',
 '##ан',
 '##ь',
 '##ш',
 '##е',
 ',',
 'ч',
 '##т',
 '##о',
 '##б',
 '##ы',
 'т',
 '##е',
 '##б',
 '##я',
 'н',
 '##е',
 'б',
 '##ы',
 '##л',
 '##о',
 'н',
 '##о',
 '##ч',
 '##ь',
 '##ю',
 '.',
 'к',
 '##а',
 '##к',
 '##о',
 '##г',
 '##о',
 'в',
 '##о',
 '##о',
 '##б',
 '##щ',
 '##е',
 'и',
 '##ии',
 '##ии',
 'т',
 '##ы',
 'и',
 '##г',
 '##р',
 '##а',
 '##е',
 '##ш',
 '##ь',
 'в',
 'м',
 '##о',
 '##е',
 '##и',
 'к',
 '##о',
 '##м',
 '##на',
 '##т',
 '##е',
 'н',
 '##о',
 '##ч',
 '##ь',
 '##ю',
 '?',
 '!',
 '&',
 'gt',
 ';',
 ':',
 '(',
 '(',
 '(',
 '[SEP]']

In [17]:
inputs = tokenizer(tokenized_texts[0], return_tensors="pt")

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length.

In [13]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")


outputs = model(**tokenized_texts, labels=train_gt)
loss = outputs.loss
logits = outputs.logits

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

TypeError: BertForSequenceClassification object argument after ** must be a mapping, not list

BERT'у нужно предоставить специальный формат входных данных.


- **input ids**: последовательность чисел, отождествляющих каждый токен с его номером в словаре.
- **labels**: вектор из нулей и единиц. В нашем случае нули обозначают негативную эмоциональную окраску, единицы - положительную.
- **segment mask**: (необязательно) последовательность нулей и единиц, которая показывает, состоит ли входной текст из одного или двух предложений. Для случая одного предложения получится вектор из одних нулей. Для двух: <length_of_sent_1> нулей и <length_of_sent_2> единиц.
- **attention mask**: (необязательно) последовательность нулей и единиц, где единицы обозначают токены предложения, нули - паддинг.


Паддинг нужен для того, чтобы BERT мог работать с предложениями разной длины. Выбираем максимально возможную длину предложения (в нашем случае пусть это будет 100). 

Теперь более длинные предложения будем обрезать до 100 токенов, а для более коротких использовать паддинг. Возьмем готовую функцию `pad_sequences` из библиотеки `keras`.

In [12]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")


outputs = model(**inputs, labels=labels)
loss = outputs.loss
logits = outputs.logits

In [14]:
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(
    input_ids, train_gt, 
    random_state=42,
    test_size=0.1
)

train_masks, validation_masks, _, _ = train_test_split(
    attention_masks,
    input_ids,
    random_state=42,
    test_size=0.1
)

Теперь подробнее рассмотрим процесс файн-тюнинга. Как мы помним, первый токен в каждом предложении - это [CLS]. В отличие от скрытого состояния, относящего к обычному слову (не метке [CLS]), скрытое состояние относящееся к этой метке должно содержать в себе аггрегированное представление всего предложения, которое дальше будет использоваться для классификации. Таким образом, когда мы скормили предложение в процессе обучения сети, выходом будет вектор со скрытым состоянием, относящийся к метке [CLS]. Дополнительный полносвязный слой, который мы добавили, имеет размер [hidden_state, количество_классов], в нашем случае количество классов равно двум. То есть нав выходе мы получим два числа, представляющих классы "положительная эмоциональная окраска" и "отрицательная эмоциональная окраска".
Процесс дообучения достаточно дешев. По факту мы тренируем наш верхний слой и немного меняем веса во всех остальных слоях в процессе, чтобы подстроиться под нашу задачу.
Иногда некоторые слои специально "замораживают" или применяют разные стратегии работы с learning rate, в общем, делают все, чтобы сохранить "хорошие" веса в нижних слоях и ускорить дообучение. В целом, замораживание слоев BERTа обычно не сильно сказывается на итоговом качестве, однако надо помнить о тех случаях, когда данные, использованные для предобучения и дообучения очень разные (разные домены или стиль: академическая и разговорная лексика). В таких случаях лучше тренировать все слои сети, не замораживая ничего.
Загружаем BERT. bert-base-uncased - это версия "base" (в оригинальной статье рассказывается про две модели: "base" vs "large"), где есть только буквы в нижнем регистре ("uncased").

In [15]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-cased')

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
preds = model.predict(validation_inputs[0:100])

In [15]:
preds = np.argmax(preds,axis=1)

In [16]:
from sklearn.metrics import accuracy_score

In [17]:
accuracy_score(preds,np.array(validation_labels[0:100]))

0.99

In [ ]:
(прототип тетрадки все еще из лекции МФТИ)